In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [2]:
pip install tensorflow_addons

In [2]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

import tensorflow_addons as tfa
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
df = pd.read_csv("colab/csv/df.csv")

In [4]:
def create_mask(image_path, json_path):
    try:
        with Image.open(image_path) as img:
            original_image = np.array(img, dtype=np.float32)
            original_image /= 255.0

        with open(json_path, 'r', encoding='UTF-8') as file:
            data = json.load(file)

        for annotation in data['labelingInfo']:
            if 'box' in annotation:
                box = annotation['box']['location'][0]
                x = box['x']
                y = box['y']
                width = box['width']
                height = box['height']

        cropped_image = original_image[y:y+height, x:x+width]

        target_size = (299, 299)
        resized_image = Image.fromarray((cropped_image * 255).astype(np.uint8))
        resized_image = resized_image.resize(target_size, Image.ANTIALIAS)

        final_image = np.array(resized_image, dtype=np.float32) / 255.0

        return final_image

    except Exception as e:
        print("error:",e)
        return np.zeros((299, 299, 3), dtype=np.float32)

In [5]:
all_keys_tensor = tf.constant(['A1', 'A2', 'A3', 'A4', 'A5', 'A6'])
all_vals_tensor = tf.constant([0, 1, 2, 3, 4, 5], dtype=tf.int64)
all_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(all_keys_tensor, all_vals_tensor), -1)

def all_map_labels(label):
    return all_table.lookup(label)

def all_prepare_dataset(image_path, json_path, label):
    mask = create_mask(image_path, json_path)

    label_index = all_map_labels(label)
    one_hot_label = tf.one_hot(label_index, depth=6)
    return mask, one_hot_label

In [6]:
def augment_image(image, label, n_augmentations=5):
    augmented_images = []
    augmented_labels = []

    augmented_images.append(image)
    augmented_labels.append(label)

    for _ in range(n_augmentations):
        # 랜덤 회전
        augmented = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

        augmented_images.append(augmented)
        augmented_labels.append(label)

    return augmented_images, augmented_labels


In [7]:
batch_size = 4

#all
all_dataset = tf.data.Dataset.from_tensor_slices((df['image_path'], df['json_path'], df['label']))
all_dataset = all_dataset.map(all_prepare_dataset, num_parallel_calls=tf.data.AUTOTUNE)

all_dataset = all_dataset.flat_map(lambda image, label: tf.data.Dataset.from_tensor_slices(augment_image(image, label)))

all_dataset_size = len(df)
all_train_size = int(0.8 * all_dataset_size)
all_val_size = all_dataset_size - all_train_size

all_dataset = all_dataset.shuffle(buffer_size=5000).cache()
all_train_dataset = all_dataset.take(all_train_size)
all_val_dataset = all_dataset.skip(all_train_size)

all_train_dataset = all_train_dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
all_val_dataset = all_val_dataset.batch(batch_size)

error: 'SymbolicTensor' object has no attribute 'read'


In [8]:
initial_learning_rate = 0.000213388
decay_rate = 1e-5
decay_steps = 1

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=1 - decay_rate,
        staircase=True)

def build_multi_model(num_classes, dropout_rate=0.5):
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None,pooling='avg',input_shape=(299, 299, 3))
    base_model.trainable = True

    x = base_model.output
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    predictions = Dense(num_classes, activation='softmax')(x)
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.999, nesterov=True)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
all_model = build_multi_model(num_classes=6)

all_callbacks = [
    EarlyStopping(monitor='val_loss',
                  patience=10,
                  verbose=1,
                  mode='min',
                  restore_best_weights=True),
    ModelCheckpoint('colab/model/all_model_{epoch:02d}.keras',
                    monitor='val_loss',
                    save_best_only=False,
                    verbose=1,
                    mode='min',
                    save_freq='epoch')
]

In [ ]:
steps_per_epoch = all_train_size // batch_size
validation_steps = all_val_size // batch_size

all_history = all_model.fit(
    all_train_dataset,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=all_val_dataset,
    validation_steps=validation_steps,
    callbacks=all_callbacks
)

Epoch 1/30
4741/6000 [======================>.......] - ETA: 4:09 - loss: 0.0442 - accuracy: 1.0000

In [ ]:
plt.figure(figsize=(18, 5))

# 훈련 및 검증 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(all_history.history['loss'], label='Training Loss')
plt.plot(all_history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 훈련 및 검증 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(all_history.history['accuracy'], label='Training Accuracy')
plt.plot(all_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()